In [1]:
from keras.models import Sequential
#Import from keras_preprocessing not from keras.preprocessing
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np
from keras.optimizers import RMSprop
import ast

traindfstart= pd.read_csv("ODIR-5K_Training_Annotations_LR.csv")
testdfstart= pd.read_csv('XYZ_ODIR.csv')


datagen=ImageDataGenerator(rescale=1./255.,validation_split=(500/3500))

2023-10-10 11:11:38.119932: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
traindf = pd.DataFrame(np.repeat(traindfstart.values, 2, axis=0))
traindf.columns = traindfstart.columns

testdf = pd.DataFrame(np.repeat(testdfstart.values, 2, axis=0))
testdf.columns = testdfstart.columns

In [10]:
traindf

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,class-left,class-right
0,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,['C'],['N']
1,0,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,['C'],['N']
2,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,['N'],['N']
3,1,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,['N'],['N']
4,2,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,"['D', 'O']",['D']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,4689,54,Male,4689_left.jpg,4689_right.jpg,mild nonproliferative retinopathy,normal fundus,0,1,0,0,0,0,0,0,['D'],['N']
6996,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0,['D'],['D']
6997,4690,57,Male,4690_left.jpg,4690_right.jpg,mild nonproliferative retinopathy,mild nonproliferative retinopathy,0,1,0,0,0,0,0,0,['D'],['D']
6998,4784,58,Male,4784_left.jpg,4784_right.jpg,hypertensive retinopathy，age-related macular d...,hypertensive retinopathy，age-related macular d...,0,0,0,0,1,1,0,0,"['A', 'H']","['A', 'H']"


In [11]:
def append_extension(fn, side):
    return f"{fn}_{side}.jpg"

# Create a new column 'Side' based on the index
traindf['Side'] = traindf.index % 2
testdf['Side'] = testdf.index % 2

# Apply the appropriate extension based on the 'Side' column
traindf['ID'] = traindf.apply(lambda row: append_extension(row['ID'], 'left' if row['Side'] == 0 else 'right'), axis=1)
traindf['labels'] = np.where(traindf['Side'] == 0, traindf['class-left'], traindf['class-right'])
traindf['Diagnostic Keywords'] = np.where(traindf['Side'] == 0, traindf['Left-Diagnostic Keywords'], traindf['Right-Diagnostic Keywords'])
# traindf['labels'] = [traindf['class-left'] if 'left' in x else traindf['class-left'] for x in traindf['ID']]

# Drop the 'Side' column if you don't need it anymore
traindf.drop(columns=['Side'], inplace=True)
testdf.drop(columns=['Side'], inplace=True)

In [12]:
traindf.head(10)

,ID,Patient Age,Patient Sex,Left-Fundus,Right-Fundus,Left-Diagnostic Keywords,Right-Diagnostic Keywords,N,D,G,C,A,H,M,O,class-left,class-right,labels,Diagnostic Keywords
0,0_left.jpg,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,['C'],['N'],['C'],cataract
1,0_right.jpg,69,Female,0_left.jpg,0_right.jpg,cataract,normal fundus,0,0,0,1,0,0,0,0,['C'],['N'],['N'],normal fundus
2,1_left.jpg,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,['N'],['N'],['N'],normal fundus
3,1_right.jpg,57,Male,1_left.jpg,1_right.jpg,normal fundus,normal fundus,1,0,0,0,0,0,0,0,['N'],['N'],['N'],normal fundus
4,2_left.jpg,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,"['D', 'O']",['D'],"['D', 'O']",laser spot，moderate non proliferative retinopathy
5,2_right.jpg,42,Male,2_left.jpg,2_right.jpg,laser spot，moderate non proliferative retinopathy,moderate non proliferative retinopathy,0,1,0,0,0,0,0,1,"['D', 'O']",['D'],['D'],moderate non proliferative retinopathy
6,3_left.jpg,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1,['N'],['O'],['N'],normal fundus
7,3_right.jpg,66,Male,3_left.jpg,3_right.jpg,normal fundus,branch retinal artery occlusion,0,0,0,0,0,0,0,1,['N'],['O'],['O'],branch retinal artery occlusion
8,4_left.jpg,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,['O'],['D'],['O'],macular epiretinal membrane
9,4_right.jpg,53,Male,4_left.jpg,4_right.jpg,macular epiretinal membrane,mild nonproliferative retinopathy,0,1,0,0,0,0,0,1,['O'],['D'],['D'],mild nonproliferative retinopathy


In [13]:
traindf[['ID', 'Patient Age', 'Patient Sex', 'labels', 'Diagnostic Keywords']]
# traindf.drop(columns=['Left-Fundus', 'Right-Fundus', 'Left-Diagnostic Keywords', 'Right-Diagnostic Keywords', 'class-left', 'class-right'], inplace=True)

,ID,Patient Age,Patient Sex,labels,Diagnostic Keywords
0,0_left.jpg,69,Female,['C'],cataract
1,0_right.jpg,69,Female,['N'],normal fundus
2,1_left.jpg,57,Male,['N'],normal fundus
3,1_right.jpg,57,Male,['N'],normal fundus
4,2_left.jpg,42,Male,"['D', 'O']",laser spot，moderate non proliferative retinopathy
...,...,...,...,...,...
6995,4689_right.jpg,54,Male,['N'],normal fundus
6996,4690_left.jpg,57,Male,['D'],mild nonproliferative retinopathy
6997,4690_right.jpg,57,Male,['D'],mild nonproliferative retinopathy
6998,4784_left.jpg,58,Male,"['A', 'H']",hypertensive retinopathy，age-related macular d...


In [8]:
traindf

,ID,Patient Age,Patient Sex,N,D,G,C,A,H,M,O,labels,Diagnostic Keywords
0,0_left.jpg,69,Female,0,0,0,1,0,0,0,0,['C'],cataract
1,0_right.jpg,69,Female,0,0,0,1,0,0,0,0,['N'],normal fundus
2,1_left.jpg,57,Male,1,0,0,0,0,0,0,0,['N'],normal fundus
3,1_right.jpg,57,Male,1,0,0,0,0,0,0,0,['N'],normal fundus
4,2_left.jpg,42,Male,0,1,0,0,0,0,0,1,"['D', 'O']",laser spot，moderate non proliferative retinopathy
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6995,4689_right.jpg,54,Male,0,1,0,0,0,0,0,0,['N'],normal fundus
6996,4690_left.jpg,57,Male,0,1,0,0,0,0,0,0,['D'],mild nonproliferative retinopathy
6997,4690_right.jpg,57,Male,0,1,0,0,0,0,0,0,['D'],mild nonproliferative retinopathy
6998,4784_left.jpg,58,Male,0,0,0,0,1,1,0,0,"['A', 'H']",hypertensive retinopathy，age-related macular d...


In [ ]:
traindf.to_csv('ODIR-5K_Training_multipleLR.csv', index=False)